In [1]:
# Import required libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# -----------------------------
# PART 1: Content-Based Filtering
# -----------------------------

# Sample movie data
movies_data = {
    'movie_id': [1, 2, 3, 4, 5],
    'title': ['Inception', 'Avengers', 'Interstellar', 'Batman', 'Iron Man'],
    'genre': ['sci-fi action', 'superhero action', 'sci-fi drama', 'superhero dark', 'superhero tech']
}

movies_df = pd.DataFrame(movies_data)
print("🎬 Movie Dataset:\n", movies_df)

# Step 1: Convert genres to TF-IDF vectors
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(movies_df['genre'])

# Step 2: Compute cosine similarity between all movies
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Step 3: Create a function to recommend similar movies based on genre
def recommend_content(movie_title, df, cosine_sim_matrix):
    # Find the index of the movie
    idx = df[df['title'] == movie_title].index[0]
    
    # Get pairwise similarity scores for that movie
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    
    # Sort movies based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get top 3 most similar movies (excluding the movie itself)
    sim_scores = sim_scores[1:4]
    
    # Get movie titles
    recommended_titles = [df.iloc[i[0]]['title'] for i in sim_scores]
    
    return recommended_titles

# Test content-based recommendation
print("\n📌 Content-Based Recommendations for 'Inception':")
print(recommend_content('Inception', movies_df, cosine_sim))

# -----------------------------
# PART 2: Collaborative Filtering
# -----------------------------

# Sample user ratings data
ratings_data = {
    'user': ['Ankit', 'Adi', 'Bhola', 'kallu', 'sona', 'annu'],
    'movie': ['sanam teri kasam', 'Devdas', 'Fanna', 'Lootera', 'Batman', 'Iron Man'],
    'rating': [5, 4, 5, 4, 5, 3]
}

ratings_df = pd.DataFrame(ratings_data)
print("\n🎟️ User Ratings Dataset:\n", ratings_df)

# Step 1: Create a user-movie rating matrix (pivot table)
user_movie_matrix = ratings_df.pivot_table(index='user', columns='movie', values='rating').fillna(0)
print("\n🎛️ User-Movie Rating Matrix:\n", user_movie_matrix)

# Step 2: Compute cosine similarity between movies (items)
movie_sim_matrix = cosine_similarity(user_movie_matrix.T)
movie_sim_df = pd.DataFrame(movie_sim_matrix, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)

# Step 3: Recommend similar movies based on ratings
def recommend_collaborative(movie_name, sim_df):
    if movie_name not in sim_df.columns:
        return ["Movie not found in ratings."]
    
    # Sort similar movies in descending order
    sim_scores = sim_df[movie_name].sort_values(ascending=False)
    
    # Exclude the movie itself and get top 3 recommendations
    recommendations = sim_scores.iloc[1:4].index.tolist()
    
    return recommendations

# Test collaborative filtering
print("\n📌 Collaborative Filtering Recommendations for 'Batman':")
print(recommend_collaborative('Batman', movie_sim_df))

# -----------------------------
# PART 3: Unified Recommendation Function
# -----------------------------

def recommend_movie(movie_title):
    print(f"\n🎥 Recommendations for '{movie_title}':")
    print("➡️ Content-Based:")
    print(recommend_content(movie_title, movies_df, cosine_sim))
    print("➡️ Collaborative Filtering:")
    print(recommend_collaborative(movie_title, movie_sim_df))

# Final test
recommend_movie('Iron Man')


🎬 Movie Dataset:
    movie_id         title             genre
0         1     Inception     sci-fi action
1         2      Avengers  superhero action
2         3  Interstellar      sci-fi drama
3         4        Batman    superhero dark
4         5      Iron Man    superhero tech

📌 Content-Based Recommendations for 'Inception':
['Interstellar', 'Avengers', 'Batman']

🎟️ User Ratings Dataset:
     user             movie  rating
0  Ankit  sanam teri kasam       5
1    Adi            Devdas       4
2  Bhola             Fanna       5
3  kallu           Lootera       4
4   sona            Batman       5
5   annu          Iron Man       3

🎛️ User-Movie Rating Matrix:
 movie  Batman  Devdas  Fanna  Iron Man  Lootera  sanam teri kasam
user                                                             
Adi       0.0     4.0    0.0       0.0      0.0               0.0
Ankit     0.0     0.0    0.0       0.0      0.0               5.0
Bhola     0.0     0.0    5.0       0.0      0.0               